# Digit recognition Notebook
This notebook should outline how the digit recognition script included in this project should work. It should outline how a Convolutional Nerual network works. It should also cover how to run the script and the different functions that it performs.

The script was trained using the MNIST dataset that was covered in a previous notebook. MNIST is a dataset that consists of 60,000 training images and labels with hand written digits in the range of 0-9 that was taken from the NIST dataset. 

## Running the script
To run the script you will need to clone the git repository from [here](https://github.com/lanodburke/Emerging-Technologies-Project.git).

This script takes in command line arguments which are outlined below: 
- -h (help): will print some instructions describing the different parameters
- -b (build): this will build the keras model that will be used to run the digit recognition
- -t (test): this will take in an input file as an argument such as a PNG, JPG or any other image format

### Build the model 
Builds the model and saves it with the .h5 file format
```zsh
python digitrec.py -b
```

### Test the model
Run the saved model and makes a prediction on the image passed in as an argument
```zsh
python digitrec.py -t image.png
```

## Convolutional Neural Networks